<a href="https://colab.research.google.com/github/lukaneco/ddwrt-db/blob/main/dd_wrt_com_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install urllib2
#!pip install beautifulsoup4
from bs4 import BeautifulSoup
#from beautifulsoup4 import BeautifulSoup
from urllib.request import urlopen
#import urllib2
import re

#html_page = urlopen("https://arstechnica.com")
#html_page = urlopen("https://www.supermercadosdia.com.ar/")
html_page = urlopen("https://dd-wrt.com/routerdb/de/routerResult.php?action=routerList")
#print(html_page.read().decode('utf-8'))
soup = BeautifulSoup(html_page)
"""
print(html_page.read().decode('utf-8'))
for link in soup.findAll('tr'):
  #<div class="product">
    #print(link.get('href'))
    #print(link)
    print(link.get('onclick'))
"""

gdp_table = soup.find("table", attrs={"class": "list"})
#print(gdp_table)


gdp_table_data = gdp_table.findAll("tr")  # contains 2 rows

#print(gdp_table_data)
# Get all the headings of Lists
headings = []
for td in gdp_table_data[0].findAll("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.text.replace('\n', ' ').strip())

print(headings)





data = {}

t_headers = []
gdp_table_data = gdp_table.findAll("tr")  # contains 2 rows
for td in gdp_table_data[0].findAll("th"):
  # remove any newlines and extra spaces from left and right
  t_headers.append(td.text.replace('\n', ' ').strip())


table_data = []
for tr in gdp_table.findAll("tr"): # find all tr's from table's tbody
    t_row = {}
    # Each table row is stored in the form of
    # t_row = {'Rank': '', 'Country/Territory': '', 'GDP(US$million)': ''}

    # find all td's(3) in tr and zip it with t_header
    for td, th in zip(tr.findAll("td"), t_headers): 

      t_row[th] = td.text.replace('\n', ' ').strip()


      #print(t_row[th])

    #print(t_row)
    if(t_row != {}):


      actionclickwithid = tr.get('onclick')

      import re
      myid = re.findall(r'\d+', actionclickwithid)[0]

      #myid = actionclickwithid.replace("showDetails('", "").replace("')", "") # showDetails('394')

      #print(myid)

      t_row["id"] = myid

      table_data.append(t_row)
      #print(t_row)

    else:
      print("empty")
      

t_headers.append("id")

# Put the data for the table with his heading.

"""
for row in table_data:
    if row:
        #print(row["id"])
        #time.sleep(1)
        #html_page = urlopen("https://dd-wrt.com/routerdb/de/routerResult.php?action=routerList")
        #print(html_page.read().decode('utf-8'))
        #soup = BeautifulSoup(html_page)
"""     


#print(t_headers)
#print(table_data)


import csv

with open("test.csv", 'w') as out_file:
    # Each 3 table has headers as following
    headers = t_headers
    writer = csv.DictWriter(out_file, headers)
    # write the header
    writer.writeheader()
    for row in table_data:
        if row:
            writer.writerow(row)



['Manufacturer', 'Model', 'Revision', 'Supported', 'Activation required']
empty


In [ ]:
#myid="402"


#print(getCssSelector("tr.info:nth-of-type(4) td[valign]"))

#router_cpu = tree.xpath('//*[@id="left"]/h1/span/span/text()')


# document.querySelector("#left > table > tbody > tr:nth-child(2) > td:nth-child(2)")

#router_ram = tree.xpath('//*[@id="left"]/table/tbody/tr[3]/td[1]/text()')

#router_flash = tree.xpath('//*[@id="left"]/table/tbody/tr[4]/td[1]/text()')




#router_additional_information = tree.xpath('//*[@id="left"]/table/tbody/tr[2]/td[3]/ul/text()')
#This will create a list of prices

#print(type(router_cpu))
#print('Name: ', router_name,router_cpu,router_ram,router_flash,router_additional_information)



def getDeviceInfo(myid):
  url =  "https://dd-wrt.com/routerdb/de/routerResult.php?action=routerDetail&id="+myid+"&site=drupal"


  print(url)
  from lxml import html

  from urllib.request import urlopen as uReq

  uClient = uReq(url)
  page_html = uClient.read()
  uClient.close()

  #page = requests.get(url)

  #print(page_html)
  #tree = html.fromstring(page_html)

  #This will create a list of buyers:
  #router_name = tree.xpath('//*[@id="left"]/h1/span/span/text()')
  from termcolor import colored

  from bs4 import BeautifulSoup as Soup
  soup = Soup(page_html)

  def getCssSelector(theselector):
    #print(colored('hello', 'red'), )
    router_cpu = soup.select(theselector)
    #print(colored(router_cpu, 'green'))
    toreturn = ""
    links = []
    
    #if(len(router_cpu) > 0):
    #toreturn = router_cpu[0].getText().replace("\n", "")

    if(len(router_cpu) > 0):
      for ici in router_cpu:

        toreturn = toreturn + ici.getText().replace("\n", "")


        thelinks = ici.findAll('a', href=True)
        if(len(thelinks) > 0):
          for link in thelinks:
            #print(colored(link['href'], 'red'))
            links.append(link['href'])
            toreturn = toreturn + " - " + link['href']
          #print(colored(links, 'blue'))
          #toreturn = toreturn + " - " + str(links)
      return toreturn
    else:
      return "null"
      


  device_info = {}
  device_info["name"] = getCssSelector("h1 span span")
  device_info["cpu"] = getCssSelector("td[width='100']")
  device_info["ram"] = getCssSelector("tr:nth-of-type(3) td[valign]")
  device_info["flash"] = getCssSelector("tr:nth-of-type(4) td[valign]")
  device_info["additional_information"] = getCssSelector(".comments")
  print(colored(device_info, 'blue'))
  return device_info

getDeviceInfo("100")

https://dd-wrt.com/routerdb/de/routerResult.php?action=routerDetail&id=100&site=drupal
{'name': 'Linksys WRT54G v6.0', 'cpu': 'BCM5352', 'ram': '8 MB', 'flash': '2 MB', 'additional_information': 'DD-WRT Wiki: Linksys WRT54G v5.0/v5.1/v6.0 - http://dd-wrt.com/wiki/index.php/Linksys_WRT54G_v5.0_%26_5.1_%26_6.0'}


{'additional_information': 'DD-WRT Wiki: Linksys WRT54G v5.0/v5.1/v6.0 - http://dd-wrt.com/wiki/index.php/Linksys_WRT54G_v5.0_%26_5.1_%26_6.0',
 'cpu': 'BCM5352',
 'flash': '2 MB',
 'name': 'Linksys WRT54G v6.0',
 'ram': '8 MB'}

# all joine

In [ ]:
#!pip install urllib2
#!pip install beautifulsoup4
from bs4 import BeautifulSoup
#from beautifulsoup4 import BeautifulSoup
from urllib.request import urlopen
#import urllib2
import re


#myid="402"


#print(getCssSelector("tr.info:nth-of-type(4) td[valign]"))

#router_cpu = tree.xpath('//*[@id="left"]/h1/span/span/text()')


# document.querySelector("#left > table > tbody > tr:nth-child(2) > td:nth-child(2)")

#router_ram = tree.xpath('//*[@id="left"]/table/tbody/tr[3]/td[1]/text()')

#router_flash = tree.xpath('//*[@id="left"]/table/tbody/tr[4]/td[1]/text()')




#router_additional_information = tree.xpath('//*[@id="left"]/table/tbody/tr[2]/td[3]/ul/text()')
#This will create a list of prices

#print(type(router_cpu))
#print('Name: ', router_name,router_cpu,router_ram,router_flash,router_additional_information)



def getDeviceInfo(myid):
  url =  "https://dd-wrt.com/routerdb/de/routerResult.php?action=routerDetail&id="+myid+"&site=drupal"


  print(url)
  from lxml import html

  from urllib.request import urlopen as uReq

  uClient = uReq(url)
  page_html = uClient.read()
  uClient.close()

  #page = requests.get(url)

  #print(page_html)
  #tree = html.fromstring(page_html)

  #This will create a list of buyers:
  #router_name = tree.xpath('//*[@id="left"]/h1/span/span/text()')
  from termcolor import colored

  from bs4 import BeautifulSoup as Soup
  soup = Soup(page_html)

  def getCssSelector(theselector):
    #print(colored('hello', 'red'), )
    router_cpu = soup.select(theselector)
    #print(colored(router_cpu, 'green'))
    toreturn = ""
    links = []
    
    #if(len(router_cpu) > 0):
    #toreturn = router_cpu[0].getText().replace("\n", "")

    if(len(router_cpu) > 0):
      for ici in router_cpu:

        toreturn = toreturn + ici.getText().replace("\n", "")


        thelinks = ici.findAll('a', href=True)
        if(len(thelinks) > 0):
          for link in thelinks:
            #print(colored(link['href'], 'red'))
            links.append(link['href'])
            toreturn = toreturn + " - " + link['href']
          #print(colored(links, 'blue'))
          #toreturn = toreturn + " - " + str(links)
      return toreturn
    else:
      return "null"
      


  device_info = {}
  device_info["name"] = getCssSelector("h1 span span")
  device_info["cpu"] = getCssSelector("td[width='100']")
  device_info["ram"] = getCssSelector("tr:nth-of-type(3) td[valign]")
  device_info["flash"] = getCssSelector("tr:nth-of-type(4) td[valign]")
  device_info["additional_information"] = getCssSelector(".comments")
  print(colored(device_info, 'blue'))
  return device_info

#getDeviceInfo("100")

#html_page = urlopen("https://arstechnica.com")
#html_page = urlopen("https://www.supermercadosdia.com.ar/")
html_page = urlopen("https://dd-wrt.com/routerdb/de/routerResult.php?action=routerList")
#print(html_page.read().decode('utf-8'))
soup = BeautifulSoup(html_page)
"""
print(html_page.read().decode('utf-8'))
for link in soup.findAll('tr'):
  #<div class="product">
    #print(link.get('href'))
    #print(link)
    print(link.get('onclick'))
"""

gdp_table = soup.find("table", attrs={"class": "list"})
#print(gdp_table)


gdp_table_data = gdp_table.findAll("tr")  # contains 2 rows

#print(gdp_table_data)
# Get all the headings of Lists
headings = []
for td in gdp_table_data[0].findAll("th"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.text.replace('\n', ' ').strip())

print(headings)





data = {}

t_headers = []
gdp_table_data = gdp_table.findAll("tr")  # contains 2 rows
for td in gdp_table_data[0].findAll("th"):
  # remove any newlines and extra spaces from left and right
  t_headers.append(td.text.replace('\n', ' ').strip())


table_data = []

import time
for tr in gdp_table.findAll("tr"): # find all tr's from table's tbody
    t_row = {}
    # Each table row is stored in the form of
    # t_row = {'Rank': '', 'Country/Territory': '', 'GDP(US$million)': ''}

    # find all td's(3) in tr and zip it with t_header
    for td, th in zip(tr.findAll("td"), t_headers): 

      t_row[th] = td.text.replace('\n', ' ').strip()


      #print(t_row[th])

    #print(t_row)
    if(t_row != {}):


      actionclickwithid = tr.get('onclick')

      import re
      myid = re.findall(r'\d+', actionclickwithid)[0]

      #myid = actionclickwithid.replace("showDetails('", "").replace("')", "") # showDetails('394')

      #print(myid)

      t_row["id"] = myid


      device_info = getDeviceInfo(myid)
      time.sleep(1)
      t_row["device_info"] = device_info

      table_data.append(t_row)
      #print(t_row)

    else:
      print("empty")
      

t_headers.append("id")

# Put the data for the table with his heading.

"""
for row in table_data:
    if row:
        #print(row["id"])
        #time.sleep(1)
        #html_page = urlopen("https://dd-wrt.com/routerdb/de/routerResult.php?action=routerList")
        #print(html_page.read().decode('utf-8'))
        #soup = BeautifulSoup(html_page)
"""     


#print(t_headers)
#print(table_data)


import csv

with open("test.csv", 'w') as out_file:
    # Each 3 table has headers as following
    headers = t_headers
    writer = csv.DictWriter(out_file, headers)
    # write the header
    writer.writeheader()
    for row in table_data:
        if row:
            writer.writerow(row)



['Manufacturer', 'Model', 'Revision', 'Supported', 'Activation required']
empty
https://dd-wrt.com/routerdb/de/routerResult.php?action=routerDetail&id=886&site=drupal
{'name': '8devices Lima - (requires activation)', 'cpu': 'QCA4531', 'ram': '64 MB', 'flash': '32 MB', 'additional_information': ''}
https://dd-wrt.com/routerdb/de/routerResult.php?action=routerDetail&id=887&site=drupal
{'name': '8devices Rambutan - (requires activation)', 'cpu': 'QCA9557', 'ram': '128 MB', 'flash': '128 MB', 'additional_information': ''}
https://dd-wrt.com/routerdb/de/routerResult.php?action=routerDetail&id=385&site=drupal
{'name': 'Abocom WAP2102 - (requires activation)', 'cpu': 'AR2317', 'ram': '16 MB', 'flash': '4 MB', 'additional_information': 'Same Hardware as : PlanexGW-MF54G2DD-WRT Forum: Flashing commands - http://www.dd-wrt.com/phpBB2/viewtopic.php?p=411384#411384'}
https://dd-wrt.com/routerdb/de/routerResult.php?action=routerDetail&id=1&site=drupal
{'name': 'Accton MR3201A - (requires activation

ValueError: ignored

In [ ]:
#print(t_headers)
#print(table_data)
import pandas as pd

datos = pd.DataFrame(table_data)
datos = datos.convert_dtypes(infer_objects=False) 
agg = datos.copy(deep=False)
  agg = agg.reset_index()

for y in agg.columns:
    print(agg[y].dtype)
    
    if(agg[y].dtype == object or type(agg[y]) == object):
        print("is obj")
        print(agg[y][0])

        #agg[y].tolist()
        # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html
        print(agg[y].name)
        agg = agg.explode(agg[y].name)
    else:
        "do nothing" 
datos.dtypes
#datos.head

IndentationError: ignored